# PhaLP

In [54]:
using MySQL
conn = DBInterface.connect(MySQL.Connection, "127.0.0.1", "root", "DB_Pw123", db = "PhaLP")

MySQL.Connection(host="127.0.0.1", user="root", port="3306", db="PhaLP")

## A look at the data

In [55]:
using DataFrames
tables = DataFrame(DBInterface.execute(conn, "SHOW TABLES;"))

Row,Tables_in_PhaLP
,String
1,CDSs
2,EC
3,UniParc
4,UniProt
5,domains
6,experimental_evidence
7,gene_ontologies
8,hosts
9,link_EC_UniProt


## Make dictionaries to map UniProt accessions to their Uniparc ID (UPI) and the other way around


In [56]:
using DataFrames
acc2upi = Dict()
upi2acc = Dict()

data = DataFrame(DBInterface.execute(conn, "SELECT * FROM PhaLP.UniProt;"))

Row,UniProt_ID,name,type,type_evidence,type_probability,UniParc_ID,phages_ID,date_created,date_last_updated
,String,String,String?,String?,Int8?,String,String,Date,Date
1,A0A023J3R2,Lysin,endolysin,ML prediction,95,UPI000009B5E8,1414741,2014-07-09,2020-10-07
2,A0A023MHA8,Endolysin,endolysin,GO annotation,99,UPI000358374C,1446490,2014-07-09,2021-09-29
3,A0A023MHL5,Lysozyme,endolysin,Protein name annotation,98,UPI00025EA4B0,1446489,2014-07-09,2021-09-29
4,A0A023NGE0,Glyco_hydro_19_cat domain-containing protein,endolysin,ML prediction,95,UPI000444C4BC,1472912,2014-07-09,2020-10-07
5,A0A023NGU6,Putative tail structure protein,VAL,ML prediction,55,UPI000444E028,1472912,2014-07-09,2021-09-29
6,A0A023VTC7,Endolysin,endolysin,GO annotation,98,UPI000456C488,1492784,2014-07-09,2021-09-29
7,A0A023W6J3,SLT domain-containing protein,endolysin,ML prediction,61,UPI000456AE9B,1481112,2014-07-09,2020-08-12
8,A0A023W6J8,Lysozyme,endolysin,ML prediction,88,UPI000456A579,1481112,2014-07-09,2021-09-29
9,A0A023W6M9,Lysozyme,VAL,InterPro domain annotation,90,UPI000456BD43,1481112,2014-07-09,2021-09-29


In [57]:
s = size(data,1)
for row in 1:s
    acc2upi[data[row, 1]]=data[row, 6]
    if data[row, 6] ∉ collect(keys(upi2acc))
        upi2acc[data[row, 6]]=[]
    end
    push!(upi2acc[data[row, 6]], data[row, 1])
end
UP2type = Dict(zip(data[:, 1], data[:, 3]))

Dict{String, Union{Missing, String}} with 17356 entries:
  "A0A7G9VVR8" => "endolysin"
  "A0A2H4YFQ7" => "endolysin"
  "A0A2S1GLL8" => "endolysin"
  "A0A7R8MRQ0" => "endolysin"
  "A0A482MER4" => "endolysin"
  "A0A7S5QZ98" => "endolysin"
  "A0A0N6WMR3" => "endolysin"
  "A0A2H4P6Z6" => "VAL"
  "A0A649VC87" => "endolysin"
  "I6QAD5"     => "VAL"
  "A0A0P0HS42" => "endolysin"
  "A0A1Z1LYD6" => "VAL"
  "A0A5J6CUF4" => "VAL"
  "A0A0B5A6U7" => "endolysin"
  "A0A514A7X8" => "VAL"
  "P19897"     => "VAL"
  "A0A097J3G5" => "endolysin"
  "A0A345L2B1" => "VAL"
  "A0A3S5H0L0" => "VAL"
  ⋮            => ⋮

In [58]:
domacc2domname = Dict()
using CSV

CSV.write("t.csv", DBInterface.execute(conn,"SELECT * FROM `PhaLP`.`link_UniParc_domains` as l join `PhaLP`.`domains` as d where d.`domains_ID` = l.`domains_ID`;"))
data = CSV.read("t.csv", DataFrame)
upi2doms = Dict()

s = size(data,1)
for row in 1:s
    if data[row,1] ∉ collect(keys(domacc2domname))
         domacc2domname[data[row,1]]=  data[row,8]
    end
    if data[row,2] ∉ collect(keys(upi2doms))
        upi2doms[data[row,2]] = []
    end
    push!(upi2doms[data[row,2]], data[row,1])
end

In [59]:
data = DataFrame(DBInterface.execute(conn,"SELECT * FROM PhaLP.UniParc;"))
seq = Dict(zip(data[:, 1],data[:,2]))



Dict{String, Vector{UInt8}} with 11549 entries:
  "UPI000178DD30" => [0x4d, 0x41, 0x4b, 0x56, 0x51, 0x46, 0x54, 0x4b, 0x52, 0x5…
  "UPI0013EFDC93" => [0x4d, 0x47, 0x59, 0x4d, 0x59, 0x50, 0x56, 0x50, 0x4b, 0x4…
  "UPI000172D062" => [0x4d, 0x56, 0x49, 0x4d, 0x53, 0x45, 0x46, 0x56, 0x57, 0x4…
  "UPI001463ABBB" => [0x4d, 0x4e, 0x4c, 0x53, 0x41, 0x4e, 0x46, 0x53, 0x4c, 0x4…
  "UPI000232F56D" => [0x4d, 0x41, 0x4c, 0x47, 0x41, 0x50, 0x4d, 0x45, 0x4e, 0x4…
  "UPI0011625D30" => [0x4d, 0x45, 0x47, 0x47, 0x44, 0x4b, 0x4c, 0x51, 0x47, 0x4…
  "UPI0009882324" => [0x4d, 0x44, 0x49, 0x44, 0x52, 0x4e, 0x52, 0x4c, 0x52, 0x5…
  "UPI000CA1D611" => [0x4d, 0x4b, 0x51, 0x41, 0x41, 0x59, 0x47, 0x53, 0x4c, 0x5…
  "UPI0006BC2F8A" => [0x4d, 0x53, 0x46, 0x4b, 0x4d, 0x4b, 0x59, 0x50, 0x49, 0x4…
  "UPI000BBF7878" => [0x4d, 0x54, 0x45, 0x52, 0x56, 0x4c, 0x52, 0x59, 0x44, 0x4…
  "UPI00138B2696" => [0x4d, 0x51, 0x4c, 0x53, 0x52, 0x4b, 0x47, 0x4c, 0x44, 0x4…
  "UPI00025D6AED" => [0x4d, 0x53, 0x4e, 0x52, 0x4e, 0x49, 0x5